In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, AncillaRegister, IBMQ
from qiskit.circuit.library import QFT, GraphState, GroverOperator
from qiskit.qasm import pi
from src.utils import *


from src.benchmarks import grover, qaoa, ghz, qft, qft_entangled, shor, w_state, qpe_inexact, qpe_exact, graph_state, vqe, deutsch_jozsa, hhl, ae

In [2]:
provider = IBMQ.load_account()
backend = provider.backend.ibmq_santiago
c_map = backend.configuration().coupling_map
max_n_arch = max(max(c_map))

In [3]:
basis_gates = backend.configuration().basis_gates
basis_gates

['id', 'rz', 'sx', 'x', 'cx', 'reset']

In [4]:
benchmarks = [grover, qaoa, ghz, qft, qft_entangled, w_state, qpe_inexact, qpe_exact, vqe, deutsch_jozsa, hhl, shor]

In [5]:
num_qubits_test = 5
n_min = 5
n_max = 10
opt_level = 2

In [11]:
characteristics = []
for benchmark in benchmarks:
    if benchmark == shor or benchmark == (hhl):
        continue
    for n in range(n_min, n_max):  
        qc = benchmark.create_circuit(n)
        
        compiled_without_architecure = get_compiled_circuit(qc = qc, opt_level=opt_level, basis_gates=basis_gates)
        save_as_qasm(compiled_without_architecure, n)       
        
        filename = qc.name + "_transpiled_" + str(n) 
        depth = compiled_without_architecure.depth()
        
        characteristics.append([filename, n, depth])
        
        if (len(compiled_without_architecure.qubits) < max_n_arch):
            compiled_with_architecture = get_compiled_circuit(qc = qc, opt_level=opt_level, 
                                                              basis_gates=basis_gates, c_map=c_map)
            save_as_qasm(compiled_with_architecture, n, True)
            
            filename = qc.name + "_transpiled_mapped_" + str(n)
            depth = compiled_with_architecture.depth()
            characteristics.append([filename, n, depth])

#create_shor_benchmarks()
#create_hhl_benchmarks

In [12]:
for row in (characteristics):
    print(row)

['grover_transpiled_5', 5, 313]
['grover_transpiled_6', 6, 908]
['grover_transpiled_7', 7, 2806]
['grover_transpiled_8', 8, 7580]
['grover_transpiled_9', 9, 22888]
['QAOA_transpiled_5', 5, 21]
['QAOA_transpiled_6', 6, 18]
['QAOA_transpiled_7', 7, 18]
['QAOA_transpiled_8', 8, 18]
['QAOA_transpiled_9', 9, 18]
['ghz_transpiled_5', 5, 8]
['ghz_transpiled_6', 6, 9]
['ghz_transpiled_7', 7, 10]
['ghz_transpiled_8', 8, 11]
['ghz_transpiled_9', 9, 12]
['qft_transpiled_5', 5, 34]
['qft_transpiled_6', 6, 41]
['qft_transpiled_7', 7, 48]
['qft_transpiled_8', 8, 55]
['qft_transpiled_9', 9, 62]
['qft_entangled_transpiled_5', 5, 42]
['qft_entangled_transpiled_6', 6, 50]
['qft_entangled_transpiled_7', 7, 58]
['qft_entangled_transpiled_8', 8, 66]
['qft_entangled_transpiled_9', 9, 74]
['w_state_transpiled_5', 5, 21]
['w_state_transpiled_6', 6, 25]
['w_state_transpiled_7', 7, 29]
['w_state_transpiled_8', 8, 33]
['w_state_transpiled_9', 9, 37]
['qpe_inexact_transpiled_5', 5, 42]
['qpe_inexact_transpiled_6'

In [13]:
def create_shor_benchmarks():
    small = [15,4] #18 qubits
    medium = [821, 4] #42 qubits
    large = [11777, 4] #58 qubits
    xlarge = [201209, 4] # 74 qubits
    benchmark_list = [small]#, medium, large, xlarge]
    for n, a in benchmark_list:         
        qc = shor.create_circuit(n,a)
        compiled_without_architecure = get_compiled_circuit(qc = qc, opt_level=opt_level, basis_gates=basis_gates)
        num_qubits = len(compiled_without_architecure.qubits)
        save_as_qasm(compiled_without_architecure, num_qubits)
        
        filename = qc.name + "_transpiled_" + str(num_qubits)
        depth = compiled_without_architecure.depth()        
        characteristics.append([filename, num_qubits, depth])
        
        if (num_qubits < max_n_arch):
            compiled_with_architecture = get_compiled_circuit(qc = qc, opt_level=opt_level, 
                                                              basis_gates=basis_gates, c_map=c_map)
            save_as_qasm(compiled_with_architecture, num_qubits, True)
            
            filename = qc.name + "_transpiled_mapped_" + str(num_qubits)
            depth = compiled_with_architecture.depth()
            characteristics.append([filename, num_qubits, depth])
create_shor_benchmarks()

In [14]:
def create_hhl_benchmarks():
    # n is not the number of qubits in this case
    for n in range(1,4):         
        qc = hhl.create_circuit(n)
        compiled_without_architecure = get_compiled_circuit(qc = qc, opt_level=opt_level, basis_gates=basis_gates)
        num_qubits = len(compiled_without_architecure.qubits)
        save_as_qasm(compiled_without_architecure, num_qubits)        
        
        filename = qc.name + "_transpiled_" + str(num_qubits)
        depth = compiled_without_architecure.depth()        
        characteristics.append([filename, num_qubits, depth])
        
        if (num_qubits < max_n_arch):
            compiled_with_architecture = get_compiled_circuit(qc = qc, opt_level=opt_level, 
                                                              basis_gates=basis_gates, c_map=c_map)
            save_as_qasm(compiled_with_architecture, num_qubits, True)
            
            filename = qc.name + "_transpiled_mapped_" + str(num_qubits)
            depth = compiled_with_architecture.depth()
            characteristics.append([filename, num_qubits, depth])
create_hhl_benchmarks()

In [15]:
for row in (characteristics):
    print(row)

['grover_transpiled_5', 5, 313]
['grover_transpiled_6', 6, 908]
['grover_transpiled_7', 7, 2806]
['grover_transpiled_8', 8, 7580]
['grover_transpiled_9', 9, 22888]
['QAOA_transpiled_5', 5, 21]
['QAOA_transpiled_6', 6, 18]
['QAOA_transpiled_7', 7, 18]
['QAOA_transpiled_8', 8, 18]
['QAOA_transpiled_9', 9, 18]
['ghz_transpiled_5', 5, 8]
['ghz_transpiled_6', 6, 9]
['ghz_transpiled_7', 7, 10]
['ghz_transpiled_8', 8, 11]
['ghz_transpiled_9', 9, 12]
['qft_transpiled_5', 5, 34]
['qft_transpiled_6', 6, 41]
['qft_transpiled_7', 7, 48]
['qft_transpiled_8', 8, 55]
['qft_transpiled_9', 9, 62]
['qft_entangled_transpiled_5', 5, 42]
['qft_entangled_transpiled_6', 6, 50]
['qft_entangled_transpiled_7', 7, 58]
['qft_entangled_transpiled_8', 8, 66]
['qft_entangled_transpiled_9', 9, 74]
['w_state_transpiled_5', 5, 21]
['w_state_transpiled_6', 6, 25]
['w_state_transpiled_7', 7, 29]
['w_state_transpiled_8', 8, 33]
['w_state_transpiled_9', 9, 37]
['qpe_inexact_transpiled_5', 5, 42]
['qpe_inexact_transpiled_6'

Grover Test

In [ ]:
a = grover.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)

QAOA Test

In [ ]:
a = qaoa.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)

GHZ Test

In [ ]:
a = ghz.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)

QFT Test

In [ ]:
a = qft.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)

QFT Entangled Test

In [ ]:
a = qft_entangled.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)

W-State Test

In [ ]:
a = w_state.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)

QPE Exact and QPE Inexact Test

In [ ]:
a = qpe_exact.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)

a = qpe_inexact.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)

Graph State Test

In [ ]:
a = graph_state.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)


Shor Test

In [ ]:
a = shor.create_circuit(15)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, None)
#c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
#save_as_qasm(c, None, True)

AE Test

In [ ]:
a = ae.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)


DJ Test

In [ ]:
a = deutsch_jozsa.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)

HHL Test

In [ ]:
import src.benchmarks.hhl
num_qubits_test = 3
a = hhl.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, 2)
#c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
#save_as_qasm(c, 2, True)
b.draw()

In [ ]:
from qiskit.algorithms.factorizers import Shor
qc = Shor().construct_circuit(201209, 4, measurement=True)
len(qc.qubits)


In [ ]:
small = [15,4] #18 qubits
medium = [821, 4] #42 qubits
large = [11777, 4] #58 qubits
xlarge = [201209, 4] # 74 qubits